# Person

> Defines a person class to be used when modelling the outbreak of a virus among people

In [1]:
#| default_exp Person

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#pip install -Uqq mesa

In [4]:
#| export
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
import random

In [5]:
#| export
class Person(Agent):
    '''A person who can become infected or immune'''
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.infected = False
        self.immune = False

    def move(self):
        '''Move the person to a neighboring cell'''
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False
        )
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def infect(self):
        '''Infect the person'''
        self.infected = True

    def step(self):
        '''Advance the person by one step in time'''
        self.move()
        if self.infected:
            self.immune = True
            self.infected = False

In [6]:
#| hide
import nbdev; nbdev.nbdev_export()